In [1]:
import requests
import pandas as pd
import datetime

In [2]:
# read file with candidate info (for this we need their name and instagram handle or id)
cand = pd.read_csv('candidates.csv')
cand

,Name,Occupation,Registration,Public Records,Tags,IG,Twitter,IG_ID
0,Floyd Bonner,Shelby County Sheriff (2018-),Democratic,Only regular publication is fluffy https://www...,county official,NaN,NaN,NaN
1,Joe Brown,"Host of Judge Joe Brown (1998-2013), former Sh...",Independent,NaN,county official,NaN,@JudgeJoeBrownTV,NaN
2,Karen Camper,Minority Leader of the Tennessee House of Repr...,Democratic,"Rep from 2008, minority leader from 2018. Been...",state official,@karendcamper,NaN,1.048210e+10
3,Frank Colvett,city councilor (2016-),Republican,PM: What council records are useful? Typically...,city official,NaN,@ColvettFrank,NaN
4,J.W. Gibson,Shelby County commissioner (2006-2010),?,https://www.shelbycountytn.gov/1218/Archived-M...,"county official, lower-priority",NaN,NaN,NaN
5,Willie Herenton,mayor (1992-2009),Democratic,NaN,"city official, lower-priority",NaN,NaN,NaN
6,Michelle McKissack,Shelby County Board of Education (2018-),Democratic,NaN,county official,@michellemckissack,NaN,1.686606e+09
7,Van Turner,Shelby County Commissioner (2014-22),Democratic,NaN,county official,@vanturner4mayor,NaN,4.478527e+08
8,Paul Young,president and CEO of the Downtown Memphis Comm...,Democratic,https://downtownmemphis.com/business/paul-youn...,city official,@youngformemphis,@youngformemphis,6.590826e+09
9,James Harvey,Shelby county commissioner (2006-18),?,NaN,"county official, lower-priority",NaN,NaN,NaN


In [3]:
# one of the research standards put forth by the client was to look into candidate activity before this date
before = datetime.datetime(2022, 5, 23)

In [4]:
# the client wanted to limit the candidate activity to about ten years
after = datetime.datetime(2012,5,23)

In [5]:
# create empty dataframe with intended columns in order
df = pd.DataFrame(columns={'Candidate','Caption','Timestamp','Shortcode'})[['Candidate','Caption','Timestamp','Shortcode']]

In [6]:
# recursive function; takes in requests object, Instagram id, and candidate name as parameters
def add_df(r, iden, candidate):
    # iterate through each post in the request page
    for e in r.json()['data']['user']['edge_owner_to_timeline_media']['edges']:
        if len(e['node']['edge_media_to_caption']['edges']) > 0:
            caption = e['node']['edge_media_to_caption']['edges'][0]['node']['text']
        else:
            caption = "None"
        timestamp = e['node']['taken_at_timestamp']
        post = datetime.datetime.fromtimestamp(timestamp)
        #if (post <= before and post >= after):
        df.loc[len(df)] = [candidate, caption ,e['node']['taken_at_timestamp'],e['node']['shortcode']]
    # if there are multiple pages of data, calls itself again using end cursor
    if r.json()['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']:
        next_page =  r.json()['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        add_df(requests.get('https://instagram.com/graphql/query/?query_id=17888483320059182&variables={"id":%s,"first":100,"after":"%s"}' % (iden, next_page)),iden,candidate)

In [7]:
for i in range(len(cand)):
    row = cand.loc[i] 
    try: 
        if '@' in row['IG']:
            # old way of getting the Instagram id, but I found that the limit was being reached much faster
            # so I pulled the Instagram ids manually and added them as a column in the candidates csv
            #user = requests.get("https://www.instagram.com/web/search/topsearch/?query=" + row['IG'][1:]).json()
            #iden = user['users'][0] ['user']['pk']
            iden = int(row['IG_ID'])
            r = requests.get('https://instagram.com/graphql/query/?query_id=17888483320059182&variables={"id":%s,"first":20,"after":null}' % iden)
            candidate = row['Name']
            add_df(r, iden, candidate)
    # certain exceptions indicate the request limit has been reached and we have to try again later
    except Exception as e:
        print(e)

argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable
argument of type 'float' is not iterable


In [8]:
df

,Candidate,Caption,Timestamp,Image,Shortcode
0,Karen Camper,Join me at this powerful event on wealth build...,1677609141,https://scontent-iad3-2.cdninstagram.com/v/t51...,CpN1vcNOAiD
1,Karen Camper,None,1668147197,https://scontent-iad3-1.cdninstagram.com/v/t51...,Ckz2g-qu27_
2,Karen Camper,None,1667877625,https://scontent-iad3-2.cdninstagram.com/v/t51...,Ckr0WNoObun
3,Karen Camper,Thanks President Biden for bringing down fuel ...,1667400632,https://scontent-iad3-2.cdninstagram.com/v/t51...,CkdmjjcubKU
4,Karen Camper,Please keep the family of Rep Barbara Cooper i...,1667089022,https://scontent-iad3-1.cdninstagram.com/v/t51...,CkUUNLZubhU
...,...,...,...,...,...
1098,Reggie Hall,None,1681861416,https://scontent-iad3-1.cdninstagram.com/v/t51...,CrMkT7NOeRN
1099,Reggie Hall,None,1679934727,https://scontent-iad3-2.cdninstagram.com/v/t51...,CqTJcXbun-W
1100,Reggie Hall,None,1679931087,https://scontent-iad3-1.cdninstagram.com/v/t51...,CqTCgCtOhrk
1101,Reggie Hall,Whitehaven you’re up next. Please join us as w...,1678207881,https://scontent-iad3-2.cdninstagram.com/v/t51...,Cpfrv2JOyv3


In [9]:
df.to_csv('ig posts.csv')